In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Thesis

Mounted at /content/drive
/content/drive/MyDrive/Thesis


In [2]:
archaic = []
modern = []
with open("dataset/zuozhuan/source.txt", "r", encoding="utf-8") as f:
    for line in f:
        archaic.append(line.strip())
with open("dataset/zuozhuan/target.txt", "r", encoding="utf-8") as f:
    for line in f:
        modern.append(line.strip())

length = len(archaic)
print(length)
print(len(modern))

15048
15048


In [3]:
print(archaic[0])
print(modern[0])

七年春，齐人伐郑。
七年春季，齐国攻打郑国。


In [4]:
all_pos = []
all_morph = []
with open("zuozhuan_pos.txt", "r", encoding="utf-8") as f:
    for line in f:
      line = line.strip().replace("[", "").replace("]", "").replace("'", "").replace(" ", "")
      pos_of_sent = []
      for pos in line.split(","):
        pos_of_sent.append(pos)
      all_pos.append(pos_of_sent)

with open("zuozhuan_morph.txt", "r", encoding="utf-8") as f:
    for line in f:
      line = line.strip().replace("[", "").replace("]", "").replace("'", "").replace(" ", "")
      morph_of_sent = []
      for morph in line.split(","):
        morph_of_sent.append(morph)
      all_morph.append(morph_of_sent)

print(len(all_pos))
print(len(all_morph))

15048
15048


In [5]:
print(archaic[526])
print(all_pos[526])
print(all_morph[526])
print(len(all_pos[526]))
print(len(all_morph[526]))
print(len(archaic[526]))

楚人伐宋以救郑。
['PROPN', 'NOUN', 'VERB', 'PROPN', 'VERB', 'VERB', 'PROPN', 'PUNCT']
['Case=Loc|NameType=Nat', '', '', 'Case=Loc|NameType=Nat', '', '', 'Case=Loc|NameType=Nat', '']
8
8
8


# Create POS and MORPH embeddings

In [6]:
from collections import defaultdict

pos_lookup = defaultdict(lambda: len(pos_lookup))
morph_lookup = defaultdict(lambda: len(morph_lookup))

pos_embeddings = []
morph_embeddings = []

for pos_seq, morph_seq in zip(all_pos, all_morph):
    pos_ids = [pos_lookup[pos] for pos in pos_seq]
    pos_ids = [101] + pos_ids + [102]
    morph_ids = [morph_lookup[morph] for morph in morph_seq]
    morph_ids = [101] + morph_ids + [102]
    pos_embeddings.append(pos_ids)
    morph_embeddings.append(morph_ids)

In [7]:
pos_vocab_size = len(pos_lookup) + 2
morph_vocab_size = len(morph_lookup) + 2

In [8]:
print(len(pos_embeddings))

print(len(morph_embeddings))

15048
15048


# New Bart Model


In [9]:
import torch
import torch.nn as nn
from transformers import AutoConfig

In [10]:
from transformers import AutoTokenizer, BertTokenizer, BartForConditionalGeneration, Text2TextGenerationPipeline, BartConfig
# tokenizer = BertTokenizer.from_pretrained("fnlp/bart-base-chinese")
tokenizer = BertTokenizer.from_pretrained("RUCAIBox/Erya")
# tokenizer = AutoTokenizer.from_pretrained("Jihuai/bert-ancient-chinese")
config = BartConfig.from_pretrained("fnlp/bart-base-chinese")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

In [12]:
from transformers.models.bart.modeling_bart import *

In [13]:
from transformers.modeling_attn_mask_utils import (
    _prepare_4d_attention_mask,
    _prepare_4d_attention_mask_for_sdpa,
    _prepare_4d_causal_attention_mask,
    _prepare_4d_causal_attention_mask_for_sdpa,
)

In [14]:
import random
class CustomBartEncoder(BartEncoder):
    def __init__(self, config: BartConfig, embed_tokens: Optional[nn.Embedding] = None):
        super().__init__(config)
        self.dropout = config.dropout
        self.layerdrop = config.encoder_layerdrop

        embed_dim = config.d_model
        self.padding_idx = config.pad_token_id
        self.max_source_positions = config.max_position_embeddings
        self.embed_scale = math.sqrt(embed_dim) if config.scale_embedding else 1.0

        self.embed_tokens = nn.Embedding(config.vocab_size, embed_dim, self.padding_idx)

        if embed_tokens is not None:
            self.embed_tokens.weight = embed_tokens.weight

        self.embed_positions = BartLearnedPositionalEmbedding(
            config.max_position_embeddings,
            embed_dim,
        )
        self.layers = nn.ModuleList([BartEncoderLayer(config) for _ in range(config.encoder_layers)])
        self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
        self._use_sdpa = config._attn_implementation == "sdpa"
        self.layernorm_embedding = nn.LayerNorm(embed_dim)

        self.gradient_checkpointing = False
        # Initialize weights and apply final processing
        self.post_init()
        self.pos_embedding = nn.Embedding(256,embed_dim,0)  # 添加的新embeddings
        self.morph_embedding = nn.Embedding(256,embed_dim,0)

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        pos_ids = None, # 新的輸入特徵
        morph_ids = None, # 新的輸入特徵
    ) -> Union[Tuple, BaseModelOutput]:

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # retrieve input_ids and inputs_embeds
        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            input = input_ids
            input_ids = input_ids.view(-1, input_ids.shape[-1])
        elif inputs_embeds is not None:
            input = inputs_embeds[:, :, -1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        if inputs_embeds is None:
            inputs_embeds = self.embed_tokens(input_ids) * self.embed_scale

        embed_pos = self.embed_positions(input)
        embed_pos = embed_pos.to(inputs_embeds.device)

        pos_embedding = self.pos_embedding(pos_ids)
        morph_embedding = self.morph_embedding(morph_ids)

        hidden_states = inputs_embeds + embed_pos + pos_embedding + morph_embedding # 全部加在一起
        hidden_states = self.layernorm_embedding(hidden_states)
        hidden_states = nn.functional.dropout(hidden_states, p=self.dropout, training=self.training)

        # expand attention_mask
        if attention_mask is not None:
            if self._use_flash_attention_2:
                attention_mask = attention_mask if 0 in attention_mask else None
            elif self._use_sdpa and head_mask is None and not output_attentions:
                # output_attentions=True & head_mask can not be supported when using SDPA, fall back to
                # the manual implementation that requires a 4D causal mask in all cases.
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                attention_mask = _prepare_4d_attention_mask_for_sdpa(attention_mask, inputs_embeds.dtype)
            else:
                # [bsz, seq_len] -> [bsz, 1, tgt_seq_len, src_seq_len]
                attention_mask = _prepare_4d_attention_mask(attention_mask, inputs_embeds.dtype)

        encoder_states = () if output_hidden_states else None
        all_attentions = () if output_attentions else None

        # check if head_mask has a correct number of layers specified if desired
        if head_mask is not None:
            if head_mask.size()[0] != (len(self.layers)):
                raise ValueError(
                    f"The head_mask should be specified for {len(self.layers)} layers, but it is for"
                    f" {head_mask.size()[0]}."
                )

        for idx, encoder_layer in enumerate(self.layers):
            if output_hidden_states:
                encoder_states = encoder_states + (hidden_states,)
            # add LayerDrop (see https://arxiv.org/abs/1909.11556 for description)
            to_drop = False
            if self.training:
                dropout_probability = torch.rand([])
                if dropout_probability < self.layerdrop:  # skip the layer
                    to_drop = True

            if to_drop:
                layer_outputs = (None, None)
            else:
                if self.gradient_checkpointing and self.training:
                    layer_outputs = self._gradient_checkpointing_func(
                        encoder_layer.__call__,
                        hidden_states,
                        attention_mask,
                        (head_mask[idx] if head_mask is not None else None),
                        output_attentions,
                    )
                else:
                    layer_outputs = encoder_layer(
                        hidden_states,
                        attention_mask,
                        layer_head_mask=(head_mask[idx] if head_mask is not None else None),
                        output_attentions=output_attentions,
                    )

                hidden_states = layer_outputs[0]

            if output_attentions:
                all_attentions = all_attentions + (layer_outputs[1],)

        if output_hidden_states:
            encoder_states = encoder_states + (hidden_states,)

        if not return_dict:
            return tuple(v for v in [hidden_states, encoder_states, all_attentions] if v is not None)
        return BaseModelOutput(
            last_hidden_state=hidden_states, hidden_states=encoder_states, attentions=all_attentions
        )

In [15]:
from transformers import BartModel

In [16]:
class BartWithPOSMorph(BartModel):
    def __init__(self, config: BartConfig):
        super().__init__(config)
        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        self.shared = nn.Embedding(vocab_size, config.d_model, padding_idx)
        self.encoder = CustomBartEncoder(config, self.shared)
        self.decoder = BartDecoder(config, self.shared)

        # Initialize weights and apply final processing
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        decoder_input_ids=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        encoder_outputs=None,
        past_key_values=None,
        inputs_embeds=None,
        decoder_inputs_embeds=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        pos_ids=None,  #自加的
        morph_ids=None,
    )-> Union[Tuple, Seq2SeqModelOutput]:
        # different to other models, Bart automatically creates decoder_input_ids from
        # input_ids if no decoder_input_ids are provided
        if decoder_input_ids is None and decoder_inputs_embeds is None:
            if input_ids is None:
                raise ValueError(
                    "If no `decoder_input_ids` or `decoder_inputs_embeds` are "
                    "passed, `input_ids` cannot be `None`. Please pass either "
                    "`input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`."
                )

            decoder_input_ids = shift_tokens_right(
                input_ids, self.config.pad_token_id, self.config.decoder_start_token_id
            )

        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if encoder_outputs is None:
          encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
                pos_ids=pos_ids, # 自加的，傳給 Encoder
                morph_ids=morph_ids, # 傳給 Encoder
            )
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )

        # decoder outputs consists of (dec_features, past_key_value, dec_hidden, dec_attn)
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            encoder_hidden_states=encoder_outputs[0],
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if not return_dict:
            return decoder_outputs + encoder_outputs

        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )

In [17]:
class CustomBartForConditionalGeneration(BartForConditionalGeneration):
    base_model_prefix = "model"
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight", "lm_head.weight"]
    _keys_to_ignore_on_load_missing = ["final_logits_bias"]

    def __init__(self, config: BartConfig):
        super().__init__(config)
        self.model = BartWithPOSMorph(config) # 替換成修改過後的類別
        self.register_buffer("final_logits_bias", torch.zeros((1, self.model.shared.num_embeddings)))
        self.lm_head = nn.Linear(config.d_model, self.model.shared.num_embeddings, bias=False)

        # Initialize weights and apply final processing
        self.post_init()

    # 修改 prepare_inputs_for_generation，提供新的特徵輸入
    # prepare_inputs_for_generation 定義了 Model.generate 如何提供資料
    def prepare_inputs_for_generation(
        self,
        decoder_input_ids,
        past_key_values=None,
        attention_mask=None,
        decoder_attention_mask=None,
        head_mask=None,
        decoder_head_mask=None,
        cross_attn_head_mask=None,
        use_cache=None,
        encoder_outputs=None,
        pos_ids=None,  # 提供輸入特徵給模型
        morph_ids=None,
        **kwargs
    ):
        # cut decoder_input_ids if past_key_values is used
        if past_key_values is not None:
            past_length = past_key_values[0][0].shape[2]

            # Some generation methods already pass only the last input ID
            if decoder_input_ids.shape[1] > past_length:
                remove_prefix_length = past_length
            else:
                # Default to old behavior: keep only final ID
                remove_prefix_length = decoder_input_ids.shape[1] - 1

            decoder_input_ids = decoder_input_ids[:, remove_prefix_length:]

        return {
            "input_ids": None,  # encoder_outputs is defined. input_ids not needed
            "encoder_outputs": encoder_outputs,
            "past_key_values": past_key_values,
            "decoder_input_ids": decoder_input_ids,
            "attention_mask": attention_mask,
            "decoder_attention_mask": decoder_attention_mask,
            "head_mask": head_mask,
            "decoder_head_mask": decoder_head_mask,
            "cross_attn_head_mask": cross_attn_head_mask,
            "use_cache": use_cache,  # change this to avoid caching (presumably for debugging)
            "pos_ids": pos_ids,  # 自加的
            "morph_ids": morph_ids
        }

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        decoder_head_mask: Optional[torch.Tensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[List[torch.FloatTensor]] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        pos_ids = None, # 提供輸入特徵給模型
        morph_ids = None
    ) -> Union[Tuple, Seq2SeqLMOutput]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if labels is not None:
            if use_cache:
                logger.warning("The `use_cache` argument is changed to `False` since `labels` is provided.")
            use_cache = False
            if decoder_input_ids is None and decoder_inputs_embeds is None:
                decoder_input_ids = shift_tokens_right(
                    labels, self.config.pad_token_id, self.config.decoder_start_token_id
                )

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            encoder_outputs=encoder_outputs,
            decoder_attention_mask=decoder_attention_mask,
            head_mask=head_mask,
            decoder_head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            decoder_inputs_embeds=decoder_inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            pos_ids=pos_ids, # 自加的 提供輸入特徵給模型
            morph_ids=morph_ids
        )
        lm_logits = self.lm_head(outputs[0])
        lm_logits = lm_logits + self.final_logits_bias.to(lm_logits.device)

        masked_lm_loss = None
        if labels is not None:
            labels = labels.to(lm_logits.device)
            loss_fct = CrossEntropyLoss()
            masked_lm_loss = loss_fct(lm_logits.view(-1, self.config.vocab_size), labels.view(-1))

        if not return_dict:
            output = (lm_logits,) + outputs[1:]
            return ((masked_lm_loss,) + output) if masked_lm_loss is not None else output

        return Seq2SeqLMOutput(
            loss=masked_lm_loss,
            logits=lm_logits,
            past_key_values=outputs.past_key_values,
            decoder_hidden_states=outputs.decoder_hidden_states,
            decoder_attentions=outputs.decoder_attentions,
            cross_attentions=outputs.cross_attentions,
            encoder_last_hidden_state=outputs.encoder_last_hidden_state,
            encoder_hidden_states=outputs.encoder_hidden_states,
            encoder_attentions=outputs.encoder_attentions,
        )

In [18]:
model = CustomBartForConditionalGeneration.from_pretrained('fnlp/bart-base-chinese') # 加載預訓練權重

model.safetensors:   0%|          | 0.00/561M [00:00<?, ?B/s]

Some weights of CustomBartForConditionalGeneration were not initialized from the model checkpoint at fnlp/bart-base-chinese and are newly initialized: ['model.encoder.morph_embedding.weight', 'model.encoder.pos_embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Load the data

In [ ]:
train_x = archaic[:10000]
train_y = modern[:10000]
train_pos_ids = pos_embeddings[:10000]
train_morph_ids = morph_embeddings[:10000]
print(len(train_x))
print(len(train_y))
print(len(train_pos_ids))
print(len(train_morph_ids))

dev_x = archaic[10000:12000]
dev_y = modern[10000:12000]
dev_pos_ids = pos_embeddings[10000:12000]
dev_morph_ids = morph_embeddings[10000:12000]
print(len(dev_x))
print(len(dev_y))
print(len(dev_pos_ids))
print(len(dev_morph_ids))

test_x = archaic[12000:]
test_y = modern[12000:]
test_pos_ids = pos_embeddings[12000:]
test_morph_ids = morph_embeddings[12000:]

print(len(test_x))
print(len(test_y))
print(len(test_pos_ids))
print(len(test_morph_ids))

10000
10000
10000
10000
2000
2000
2000
2000
3048
3048
3048
3048


In [ ]:
from torch.nn.utils.rnn import pad_sequence

# train_pos_ids = [torch.tensor(pos_ids) for pos_ids in train_pos_ids]
# train_pos_ids = pad_sequence(train_pos_ids, batch_first=True, padding_value=0)
# train_pos_ids = train_pos_ids[:, :256]

# train_morph_ids = [torch.tensor(morph_ids) for morph_ids in train_morph_ids]
# train_morph_ids = pad_sequence(train_morph_ids, batch_first=True, padding_value=0)
# train_morph_ids = train_morph_ids[:, :256]

In [ ]:
train_input_ids = tokenizer(train_x, return_tensors='pt', padding=True, max_length=115, truncation=True)['input_ids']
train_labels = tokenizer(train_y, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

train_pos_ids = [torch.tensor(pos_ids) for pos_ids in train_pos_ids]
train_pos_ids = pad_sequence(train_pos_ids, batch_first=True, padding_value=0)
train_pos_ids = train_pos_ids[:, :256]

train_morph_ids = [torch.tensor(morph_ids) for morph_ids in train_morph_ids]
train_morph_ids = pad_sequence(train_morph_ids, batch_first=True, padding_value=0)
train_morph_ids = train_morph_ids[:, :256]

print(train_input_ids.shape)
print(train_labels.shape)
print(train_pos_ids.shape)
print(train_morph_ids.shape)

torch.Size([10000, 115])
torch.Size([10000, 174])
torch.Size([10000, 115])
torch.Size([10000, 115])


In [ ]:
dev_input_ids = tokenizer(dev_x, return_tensors='pt', padding=True, max_length=133, truncation=True)['input_ids']
dev_labels = tokenizer(dev_y, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

dev_pos_ids = [torch.tensor(pos_ids) for pos_ids in dev_pos_ids]
dev_pos_ids = pad_sequence(dev_pos_ids, batch_first=True, padding_value=0)
dev_pos_ids = dev_pos_ids[:, :256]

dev_morph_ids = [torch.tensor(morph_ids) for morph_ids in dev_morph_ids]
dev_morph_ids = pad_sequence(dev_morph_ids, batch_first=True, padding_value=0)
dev_morph_ids = dev_morph_ids[:, :256]

print(dev_input_ids.shape)
print(dev_labels.shape)
print(dev_pos_ids.shape)
print(dev_morph_ids.shape)

torch.Size([2000, 77])
torch.Size([2000, 150])
torch.Size([2000, 77])
torch.Size([2000, 77])


In [ ]:
test_input_ids = tokenizer(test_x, return_tensors='pt', padding=True, max_length=98, truncation=True)['input_ids']
test_labels = tokenizer(test_y, return_tensors='pt', padding=True, max_length=256, truncation=True)['input_ids']

test_pos_ids = [torch.tensor(pos_ids) for pos_ids in test_pos_ids]
test_pos_ids = pad_sequence(test_pos_ids, batch_first=True, padding_value=0)
test_pos_ids = test_pos_ids[:, :104]

test_morph_ids = [torch.tensor(morph_ids) for morph_ids in test_morph_ids]
test_morph_ids = pad_sequence(test_morph_ids, batch_first=True, padding_value=0)
test_morph_ids = test_morph_ids[:, :104]

print(test_input_ids.shape)
print(test_labels.shape)
print(test_pos_ids.shape)
print(test_morph_ids.shape)

torch.Size([3048, 98])
torch.Size([3048, 144])
torch.Size([3048, 98])
torch.Size([3048, 98])


In [ ]:
from torch.utils.data import Dataset, DataLoader
class TranslationDataset(Dataset):
    def __init__(self, archaic_ids, modern_ids, pos_tags, morph_tags): #, attention_masks):
        self.archaic_ids = archaic_ids
        self.modern_ids = modern_ids
        self.pos_tags = pos_tags
        self.morph_tags = morph_tags
        # self.attention_masks = attention_masks

    def __len__(self):
        return len(self.archaic_ids)

    def __getitem__(self, index):
        archaic_id = self.archaic_ids[index]
        modern_id = self.modern_ids[index]
        pos_tag = self.pos_tags[index]
        morph_tag = self.morph_tags[index]
        # attention_mask = self.attention_masks[index]
        return archaic_id, modern_id, pos_tag, morph_tag #, attention_mask

train_dataset = TranslationDataset(train_input_ids, train_labels, train_pos_ids, train_morph_ids)#, train_attention_mask)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)

dev_dataset = TranslationDataset(dev_input_ids, dev_labels, dev_pos_ids, dev_morph_ids)#, dev_attention_mask)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=False)

test_dataset = TranslationDataset(test_input_ids, test_labels, test_pos_ids, test_morph_ids)#, test_attention_mask)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Training

load previous model for further training

In [ ]:
# checkpoint_path = 'arChSynBart_POS_MORPH_erya.pt'
# model.load_state_dict(torch.load(checkpoint_path))

In [ ]:
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 10

device = torch.device("cuda")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
!pip install wandb
from tqdm import tqdm
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00


In [ ]:
model.to(device)
wandb.init(project='bart-pos-morph', config=config)

# 配置超参数
config = wandb.config
config.num_epochs = 10
config.batch_size = 16
config.learning_rate = 1e-5

best_eval_loss = float('inf')
patience = 3
no_improvement_count = 0

# 训练模型
for epoch in range(config.num_epochs):
    model.train()
    train_loss = 0
    train_steps = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1} Training")
    for batch in progress_bar:
        input_ids, output_ids, pos_ids, morph_ids = batch
        input_ids = input_ids.to(device)
        output_ids = output_ids.to(device)
        pos_ids = pos_ids.to(device)
        morph_ids = morph_ids.to(device)

        outputs = model(input_ids=input_ids, labels=output_ids, pos_ids=pos_ids, morph_ids=morph_ids)
        logits = outputs.logits
        # loss = criterion(logits.view(-1, logits.size(-1)), output_ids[:, 1:].contiguous().view(-1))
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss += loss.item()
        train_steps += 1
        progress_bar.set_postfix({'loss': loss.item()})

    train_loss /= train_steps
    wandb.log({'epoch': epoch + 1, 'train_loss': train_loss})

    # 在验证集上评估模型
    model.eval()
    eval_loss = 0
    eval_steps = 0
    progress_bar = tqdm(dev_dataloader, desc=f"Epoch {epoch + 1} Evaluation")

    with torch.no_grad():
        for batch in progress_bar:
            input_ids, output_ids, pos_ids, morph_ids = batch
            input_ids = input_ids.to(device)
            output_ids = output_ids.to(device)
            pos_ids = pos_ids.to(device)
            morph_ids = morph_ids.to(device)

            outputs = model(input_ids=input_ids, labels=output_ids, pos_ids=pos_ids, morph_ids=morph_ids)
            logits = outputs.logits
            # loss = criterion(logits.view(-1, logits.size(-1)), output_ids[:, 1:].contiguous().view(-1))
            loss = outputs.loss

            eval_loss += loss.item()
            eval_steps += 1
            progress_bar.set_postfix({'loss': loss.item()})

            # pred = model.generate(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids, do_sample=False, num_beams=5)
            # generated_sentences.extend(tokenizer.batch_decode(pred, skip_special_tokens=True))

    eval_loss /= eval_steps
    wandb.log({'epoch': epoch + 1, 'eval_loss': eval_loss})

    if eval_loss < best_eval_loss:
        best_eval_loss = eval_loss
        no_improvement_count = 0
    else:
        no_improvement_count += 1

        if no_improvement_count >= patience:
            old_lr = config.learning_rate
            config.learning_rate *= 0.5
            print(f"No improvement in {patience} epochs. Reducing learning rate from {old_lr} to {config.learning_rate}")
            no_improvement_count = 0
            for param_group in optimizer.param_groups:
                param_group['lr'] = config.learning_rate


    # 评估生成的翻译结果
    # bleu_score = calculate_bleu(generated_sentences, dev_y)
    # wandb.log({'epoch': epoch + 1, 'bleu_score': bleu_score})

wandb.finish()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 10 Evaluation: 100%|██████████| 125/125 [00:07<00:00, 17.85it/s, loss=0.362]


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
eval_loss,█▅▄▃▂▂▁▁▁▁
train_loss,█▄▃▃▂▂▂▁▁▁
epoch,10
eval_loss,0.28033
train_loss,0.18808


# Evaluation


In [ ]:
import torch
torch.save(model.state_dict(), "/content/drive/MyDrive/Thesis/arChSynBart_POS_MORPH_trained_on_zuozhuan_erya.pt")

In [19]:
checkpoint_path = 'arChSynBart_POS_MORPH_trained_on_zuozhuan_erya.pt'
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [ ]:
!pip install evaluate
!pip install bert_score
import evaluate
bertscore = evaluate.load("bertscore")

In [ ]:
predicted_translations = []
with torch.no_grad():
  for batch in tqdm(test_dataloader, desc="Testing"):
    input_ids, _, pos_ids, morph_ids = batch
    input_ids = input_ids.to(device)
    pos_ids = pos_ids.to(device)
    morph_ids = morph_ids.to(device)

    # logits = model(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids).logits
    generated_ids = model.generate(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids, num_beams=5, do_sample=False, no_repeat_ngram_size=3, max_length=256)
    # pred_input_ids = model.generate(input_ids=input_ids, pos_ids=pos_ids, morph_ids=morph_ids, do_sample=False, num_beams=5
    predicted_translations.extend(tokenizer.batch_decode(generated_ids, skip_special_tokens=True))

score = bertscore.compute(predictions=predicted_translations, references=test_y, lang="zh")
print("Bertscore: ", score)

In [ ]:
# anchibert tokenizer + bart-base-chinese
# print(predicted_translations[1016])
# print(test_y[1016])
# print(test_x[1016])

In [ ]:
# jihuai tokenizer
# print(predicted_translations[820])
# print(test_y[820])
# print(test_x[820])

In [ ]:
# bart base chinese
# print(len(predicted_translations[11]))
# print(predicted_translations[9])
# print(test_y[9])
# print(test_x[9])

In [ ]:
# erya tokenizer
print(len(predicted_translations[526]))
print(predicted_translations[526])
print(test_y[526])
print(test_x[526])

47
武 公 、 献 公 以 下 ， 治 理 国 家 的 人 很 多 ， 谁 能 治 理 得 了 ？
武公、献公以来，兼并的国家就多了，谁能够治理它？
武、献以下，兼国多矣，谁得治之？


In [ ]:
new = {}
new['precision'] = sum(score['precision']) / len(score['precision'])
new['recall'] = sum(score['recall']) / len(score['recall'])
new['f1'] = sum(score['f1']) / len(score['f1'])
print(new)

{'precision': 0.8564342044860985, 'recall': 0.8301310881463875, 'f1': 0.8425519673295534}


In [ ]:
print(morph_lookup)

defaultdict(<function <lambda> at 0x791fff16e710>, {'': 0, 'Case=Tem': 1, 'Case=Loc|NameType=Nat': 2, 'Degree=Pos': 3, 'Case=Loc': 4, 'Person=3|PronType=Prs': 5, 'NameType=Prs': 6, 'NameType=Giv': 7, 'Case=Loc|NameType=Geo': 8, 'NameType=Sur': 9, 'AdvType=Tim': 10, 'Polarity=Neg': 11, 'AdvType=Cau': 12, 'AdvType=Tim|Aspect=Perf': 13, 'Mood=Pot': 14, 'Person=1|PronType=Prs': 15, 'PronType=Int': 16, 'AdvType=Tim|Tense=Fut': 17, 'Person=2|PronType=Prs': 18, 'Degree=Equ': 19, 'AdvType=Deg|Degree=Sup': 20, 'VerbType=Cop': 21, 'PronType=Dem': 22, 'Mood=Des': 23, 'NumType=Ord': 24, 'PronType=Prs': 25, 'PronType=Prs|Reflex=Yes': 26, 'NounType=Clf': 27, 'AdvType=Tim|Tense=Past': 28, 'AdvType=Tim|Tense=Pres': 29, 'Mood=Nec': 30, 'Voice=Pass': 31, 'AdvType=Deg|Degree=Pos': 32})


In [ ]:
# sent = tokenizer("不父乙咎妇好")['input_ids']
# po = torch.tensor([101,4,0,0,2,0,0,102])
# mo = torch.tensor([101,8,1,1,0,1,1,102])
# p = model.generate(input_ids=torch.tensor([sent]).to(device), pos_ids=po.to(device), morph_ids=mo.to(device))
# print(tokenizer.batch_decode(p, skip_special_tokens=True))

In [20]:
!pip install suparkanbun
import suparkanbun
import deplacy
# deplacy.render(doc)
# deplacy.serve(doc,port=None)
nlp=suparkanbun.load(BERT="roberta-classical-chinese-base-char",Danku=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 957.5/957.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1

(…)sical-chinese-base-char.pos/filesize.txt:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/423M [00:00<?, ?B/s]

(…)oberta-classical-chinese-base-char.supar:   0%|          | 0.00/473M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/106k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/428M [00:00<?, ?B/s]

(…)cal-chinese-base-char.danku/filesize.txt:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/423M [00:00<?, ?B/s]

In [ ]:
# sent = "君祀無乃殄乎"
# sent = tokenizer(sent)['input_ids']
# p_list = [101, 4, 3, 7, 10, 2, 13, 102]
# po = torch.tensor(p_list)
# m_list = [101, 12, 6, 13, 9, 0, 0, 102]
# mo = torch.tensor(m_list)

In [ ]:
import torch
device = 'cuda'
model.to(device)

In [74]:
sent = "未有若汝封之心"
doc=nlp(sent)
p_list = [101]
m_list = [101]
for tok in doc:
    p = pos_lookup[str(tok.pos_)]
    m = morph_lookup[str(tok.morph)]
    p_list.append(p)
    m_list.append(m)
p_list.append(102)
m_list.append(102)
print(p_list)
print(m_list)

INFO:supar:Loading the data
INFO:supar:
Dataset(n_sentences=1, n_batches=1, n_buckets=1)
INFO:supar:Making predictions on the dataset
INFO:supar:0:00:00.029379s elapsed, 34.04 Sents/s


[101, 9, 4, 4, 5, 4, 12, 1, 102]
[101, 11, 0, 19, 18, 0, 0, 0, 102]


In [75]:
# sent = "今季孙乃始血,其毋乃未可知也"
# sent = tokenizer(sent)['input_ids']
# p_list = [101, 9, 3, 3, 9, 4, 4, 2, 9, 9, 9, 9, 10, 4, 6, 102]
# po = torch.tensor(p_list)
# m_list = [101, 10, 9, 7, 12, 0, 0, 0, 12, 11, 12, 11, 14, 0, 0, 102]
# mo = torch.tensor(m_list)

# sent = "毋或如东门遂不听公命杀嫡立庶"
# p_list = [101, 4, 5, 9, 1, 1, 1, 9, 4, 1, 1,  4, 1, 4, 4, 102]
# m_list = [101, 11, 5, 19, 9, 9, 7, 11, 0, 0, 0,  0, 0, 0, 3, 102]

# sent = "丙子卜：弗其克？允不。"
# p_list = [101, 0, 0, 4, 2, 9, 5, 4, 2, 4, 9, 2, 102]
# m_list = [101, 0, 0, 0, 0, 11, 22, 0, 0, 0, 11, 0, 102]
sent = tokenizer(sent)['input_ids']
po = torch.tensor(p_list)
mo = torch.tensor(m_list)
p = model.generate(input_ids=torch.tensor([sent]).to(device), morph_ids=mo.to(device), pos_ids=po.to(device), max_length=256, num_beams=8, do_sample=True)
print(tokenizer.batch_decode(p, skip_special_tokens=True))

['还 没 有 像 汝 封 那 样 的 心 意 。']


In [ ]:
print(pos_lookup)

defaultdict(<function <lambda> at 0x791fff16e7a0>, {'NUM': 0, 'NOUN': 1, 'PUNCT': 2, 'PROPN': 3, 'VERB': 4, 'PRON': 5, 'SYM': 6, 'ADP': 7, 'CCONJ': 8, 'ADV': 9, 'AUX': 10, 'PART': 11, 'SCONJ': 12, 'INTJ': 13})


In [31]:
print(morph_lookup)
morphstypes = morph_lookup.keys()
morphsids = morph_lookup.values()
morphstypes = list(morphstypes)
morphsids = list(morphsids)
for i in range(34):
  for mor in morphstypes:
    if morph_lookup[mor] == i:
      print(i,' ',mor)

[101, 10, 9, 7, 12, 0, 0, 0, 12, 11, 12, 11, 14, 0, 0, 102]


defaultdict(<function <lambda> at 0x7c609376e7a0>, {'': 0, 'Case=Tem': 1, 'Case=Loc|NameType=Nat': 2, 'Degree=Pos': 3, 'Case=Loc': 4, 'Person=3|PronType=Prs': 5, 'NameType=Prs': 6, 'NameType=Giv': 7, 'Case=Loc|NameType=Geo': 8, 'NameType=Sur': 9, 'AdvType=Tim': 10, 'Polarity=Neg': 11, 'AdvType=Cau': 12, 'AdvType=Tim|Aspect=Perf': 13, 'Mood=Pot': 14, 'Person=1|PronType=Prs': 15, 'PronType=Int': 16, 'AdvType=Tim|Tense=Fut': 17, 'Person=2|PronType=Prs': 18, 'Degree=Equ': 19, 'AdvType=Deg|Degree=Sup': 20, 'VerbType=Cop': 21, 'PronType=Dem': 22, 'Mood=Des': 23, 'NumType=Ord': 24, 'PronType=Prs': 25, 'PronType=Prs|Reflex=Yes': 26, 'NounType=Clf': 27, 'AdvType=Tim|Tense=Past': 28, 'AdvType=Tim|Tense=Pres': 29, 'Mood=Nec': 30, 'Voice=Pass': 31, 'AdvType=Deg|Degree=Pos': 32})
0   
1   Case=Tem
2   Case=Loc|NameType=Nat
3   Degree=Pos
4   Case=Loc
5   Person=3|PronType=Prs
6   NameType=Prs
7   NameType=Giv
8   Case=Loc|NameType=Geo
9   NameType=Sur
10   AdvType=Tim
11   Polarity=Neg
12   AdvType

In [ ]:
# sent = tokenizer("今季孙乃始血，其毋乃未可知也")['input_ids']
# po = torch.tensor([101, 4, 12, 3, 1, 1, 1, 16, 8, 3, 8, 10, 1, 1, 102])
# mo = torch.tensor([101, 1, 0, 5, 2, 1, 8, 4, 5, 5, 5, 9, 2, 7, 102])
# p = model.generate(input_ids=torch.tensor([sent]).to(device), pos_ids=po.to(device), morph_ids=mo.to(device), max_length=256)
# print(tokenizer.batch_decode(p, skip_special_tokens=True))

In [ ]:
import json

In [ ]:
with open('arChSynBart-erya-tokenizer-train_and_test_on_large_bart_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write('\n the following are of erya tokenizer')
    json.dump(new, f, ensure_ascii=False, indent=4)
    f.write('\n')
    f.write('epoch=10, batch_size=16, lr=1e-5, num_beams=5\n')
    f.write(model.config.to_json_string())